In [1]:
# !git clone https://github.com/GabrielGlzSa/ModelCompression.git
# %cd ModelCompression/
# !git checkout --track origin/CompressionV2
# !pip install Pillow \
#     h5py \
#     keras_preprocessing \
#     matplotlib \
#     mock \
#     numpy \
#     scipy \
#     sklearn \
#     pandas \
#     future \
#     portpicker \
#     enum34 \
#     tensorflow==2.6.2 \
#     tensorflow_datasets==4.0.1\
#     tensorflow_transform\
#     tensorboard_plugin_profile\
#     seaborn\
#     pyparsing==2.4.7\
#     tf_agents \
#     tensorflow-model-optimization
%load_ext tensorboard

In [2]:
from google.colab import drive
import sys

drive.mount('/content/drive/')
%cd ./drive/MyDrive/Colab Projects/
%ls

sys.path.insert(0, './ModelCompression')
dataset = 'mnist'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/Colab Projects
 Adadeep.ipynb   data/                     ModelCompression/
 checkpoints/   'DQN Optimization.ipynb'  'Test compressors.ipynb'


In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import logging
import pandas as pd
from IPython.display import clear_output
from utils import load_dataset
from CompressionTechniques import *
from replay_buffer import ReplayBuffer
from environments import *
from custom_layers import ROIEmbedding, MLPConvV1
logging.basicConfig(level=logging.INFO, format='%(asctime)s -%(levelname)s - %(funcName)s -  %(message)s')

In [4]:
# kernel_size = (3,3)
# channels = 3
# bases = 4
# filters = 32
# fh, fw = kernel_size
# S = np.random.random(size=(channels, bases, filters))
# Q = np.random.random(size=(channels, fh, fw, bases))
# P = np.random.random(size=(channels, channels))
# v1 = SparseConvolution2D(kernel_size=kernel_size,
#                            filters=filters,
#                            PQS=[P, Q, S],
#                            bases=bases)

# imgs = tf.constant(np.random.randint(0, 255, size=(4,32,32,3)).astype(np.float32))
# %timeit v1(imgs)

In [5]:
def make_env(dataset):
    train_ds, val_ds, test_ds, input_shape, num_classes = load_dataset(dataset)
    train_ds, valid_ds, test_ds, input_shape, num_classes = load_dataset(dataset)

    model_path = './data/full_model/test_'+dataset
    optimizer = tf.keras.optimizers.Adam()
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
    train_metric = tf.keras.metrics.SparseCategoricalAccuracy()
    try:
      model = tf.keras.models.load_model(model_path, compile=True)
    except OSError:
      
      model = tf.keras.Sequential([tf.keras.layers.Conv2D(32, (3, 3), activation='relu', name='conv2d_0',
                                                          input_shape=input_shape),
                                   tf.keras.layers.Conv2D(32, (3, 3), activation='relu', name='conv2d_1'),
                                   tf.keras.layers.Conv2D(32, (3, 3), activation='relu', name='conv2d_2'),
                                   tf.keras.layers.MaxPool2D((2, 2), 2),
                                   tf.keras.layers.Flatten(),
                                   tf.keras.layers.Dense(128, activation='relu', name='dense_0'),
                                   tf.keras.layers.Dense(128, activation='relu', name='dense_1'),
                                   tf.keras.layers.Dense(num_classes, activation='softmax', name='dense_softmax')
                                   ])
      model.compile(optimizer=optimizer, loss=loss_object, metrics=train_metric)
      model.fit(train_ds, epochs=5, validation_data=valid_ds)
      model.save(model_path)

    parameters = {}
    parameters['DeepCompression'] = {'layer_name': 'dense_0', 'threshold': 0.001}
    parameters['ReplaceDenseWithGlobalAvgPool'] = {'layer_name': 'dense_1'}
    parameters['InsertDenseSVD'] = {'layer_name': 'dense_0', 'units': 16}
    parameters['InsertDenseSVDCustom'] = {'layer_name': 'dense_0', 'units': 16}
    parameters['InsertDenseSparse'] = {'layer_name': 'dense_0', 'verbose': True, 'units': 16}
    parameters['InsertSVDConv'] = {'layer_name': 'conv2d_1', 'units': 8}
    parameters['DepthwiseSeparableConvolution'] = {'layer_name': 'conv2d_1'}
    parameters['FireLayerCompression'] = {'layer_name': 'conv2d_1'}
    # parameters['MLPCompression'] = {'layer_name': 'conv2d_1'}
    parameters['SparseConnectionsCompression'] = {'layer_name': 'conv2d_1', 'epochs': 20,
                                                  'target_perc': 0.75, 'conn_perc_per_epoch': 0.1}
    # compressor = ReplaceDenseWithGlobalAvgPool(model=model, dataset=train_ds,
    #                                            optimizer=optimizer, loss=loss_object, metrics=train_metric)
    # compressor.compress_layer()


    # compressor = InsertDenseSVD(model=model, dataset=train_ds,
    #                             optimizer=optimizer, loss=loss_object, metrics=train_metric, fine_tune=False)
    # compressor.compress_layer('dense_1', units=32)


    # compressor = InsertDenseSparse(model=model, dataset=train_ds,
    #                             optimizer=optimizer, loss=loss_object, metrics=train_metric)
    # compressor.compress_layer('dense_1', verbose=True)


    # compressor = InsertSVDConv(model=model, dataset=train_ds,
    #                             optimizer=optimizer, loss=loss_object, metrics=train_metric)
    # compressor.compress_layer('conv2d_1')

    # compressor = DepthwiseSeparableConvolution(model=model, dataset=train_ds,
    #                             optimizer=optimizer, loss=loss_object, metrics=train_metric)
    # compressor.compress_layer('conv2d_1')

    # compressor = FireLayerCompression(model=model, dataset=train_ds,
    #                                   optimizer=optimizer, loss=loss_object, metrics=train_metric)
    # compressor.compress_layer('conv2d_1')

    # compressor = MLPCompression(model=model, dataset=train_ds,
    #                             optimizer=optimizer, loss=loss_object, metrics=train_metric)
    # compressor.compress_layer('conv2d_1')

    # population=20,generations=100 takes 3 hours and 40 minutes
    # compressor = SparseConnectionsCompressionCustom(model=model,
    #                                                 dataset=train_ds,
    #                                                 optimizer=optimizer,
    #                                                 loss=loss_object,
    #                                                 metrics=train_metric)
    # compressor.compress_layer('conv2d_1', epochs=10)

    # RAM OOM in training loop.
    compressor = SparseConvolutionCompression(model=model, dataset=train_ds,
                                optimizer=optimizer, loss=loss_object, metrics=train_metric, bases=4, input_shape=input_shape)
    compressor.compress_layer('conv2d_1', iterations=2)

    new_model = compressor.get_model()
    for layer in new_model.layers:
        print(layer)


make_env(dataset)

2022-04-07 08:17:47,441 -INFO - read_from_directory -  Load dataset info from ./data/mnist/3.0.1
2022-04-07 08:17:47,463 -INFO - read_from_directory -  Field info.citation from disk and from code do not match. Keeping the one from code.
2022-04-07 08:17:47,469 -INFO - download_and_prepare -  Reusing dataset mnist (./data/mnist/3.0.1)
2022-04-07 08:17:47,482 -INFO - as_dataset -  Constructing tf.data.Dataset for split ['train[:80%]', 'train[80%:]', 'test'], from ./data/mnist/3.0.1
2022-04-07 08:17:49,029 -INFO - read_from_directory -  Load dataset info from ./data/mnist/3.0.1
2022-04-07 08:17:49,042 -INFO - read_from_directory -  Field info.citation from disk and from code do not match. Keeping the one from code.
2022-04-07 08:17:49,049 -INFO - download_and_prepare -  Reusing dataset mnist (./data/mnist/3.0.1)
2022-04-07 08:17:49,065 -INFO - as_dataset -  Constructing tf.data.Dataset for split ['train[:80%]', 'train[80%:]', 'test'], from ./data/mnist/3.0.1
2022-04-07 08:17:50,792 -INFO 

2022-04-07 08:17:50,988 -WARNING - _fallback_converter -  Using a while_loop for converting Slice


2022-04-07 08:17:51,138 -WARNING - _fallback_converter -  Using a while_loop for converting Slice
2022-04-07 08:17:55,522 -INFO - find_pqs -  Epoch 0 of RxP Loss: 76.98979949951172


2022-04-07 08:17:55,597 -WARNING - _fallback_converter -  Using a while_loop for converting Slice


2022-04-07 08:17:55,796 -WARNING - _fallback_converter -  Using a while_loop for converting Slice
2022-04-07 08:18:00,714 -INFO - find_pqs -  Searching for matrices Q and S.


...

2022-04-07 08:18:06,336 -WARNING - _fallback_converter -  Using a while_loop for converting Slice
2022-04-07 08:18:10,992 -INFO - find_pqs -  Matrix P has shape (32, 32)
2022-04-07 08:18:11,000 -INFO - find_pqs -  Matrix Q has shape (32, 3, 3, 4)
2022-04-07 08:18:11,005 -INFO - find_pqs -  Matrix S has shape (32, 4, 32)


2022-04-07 08:18:11,886 -WARNING - _fallback_converter -  Using a while_loop for converting Slice


2022-04-07 08:18:12,145 -WARNING - _fallback_converter -  Using a while_loop for converting Slice


Epoch 1/2
  88/1500 [>.............................] - ETA: 6:42 - loss: 1.8747 - sparse_categorical_accuracy: 0.9119

KeyboardInterrupt: ignored

In [ ]:
%tensorboard --logdir data/logdir/tfboard_v1